### Automatic Vehicle Number Plate Detection

In [ ]:
import os
import time
import numpy as np
import matplotlib.pyplot as plt
import cv2
import torch
from IPython.display import Image

In [ ]:
!git clone https://github.com/ultralytics/yolov5

In [ ]:
%cd yolov5
%pip install setuptools --upgrade
%pip install -r requirements.txt

In [ ]:
img_path = "VOC2007/JPEGImages/"
ann_path = "VOC2007/Annotations/"
trainvalfile = "VOC2007/ImageSets/Main/trainval.txt"
testfile = "VOC2007/ImageSets/Main/test.txt"

In [ ]:
from xml.etree import ElementTree
lab_path = "VOC2007/COCOLabels/"
try:
    os.mkdir(lab)
except Exception:
    pass
for f in os.listdir(ann_path):
    tree = ElementTree.parse(os.path.join(ann_path, f))
    ann = tree.getroot()
    x1 = float(ann[6][4][0].text)
    y1 = float(ann[6][4][1].text)
    x2 = float(ann[6][4][2].text)
    y2 = float(ann[6][4][3].text)
    width = float(ann[4][0].text)
    height = float(ann[4][1].text)
    x = (x1+x2)/(2*width)
    y = (y1+y2)/(2*height)
    w = (x2-x1)/width
    h = (y2-y1)/height
    l = os.path.splitext(f)[0]
    labels = open("VOC2007/COCOLabels/"+l+".txt", 'w')
    labels.write("0 {:.6f} {:.6f} {:.6f} {:.6f}".format(x, y, w, h))
    labels.close()

In [ ]:
import shutil
train_img = "yolov5/data/number_plates_dataset/images/train"
val_img = "yolov5/data/number_plates_dataset/images/val"
test_img = "yolov5/data/number_plates_dataset/images/test"
train_lab = "yolov5/data/number_plates_dataset/labels/train"
val_lab = "yolov5/data/number_plates_dataset/labels/val"
try:
    os.mkdir(train_img)
    os.mkdir(val_img)
    os.mkdir(test_img)
    os.mkdir(train_lab)
    os.mkdir(val_lab)
except Exception:
    pass
trainvaltxt = open(trainvalfile, 'r')
testtxt = open(testfile, 'r')
trainvallines = trainvaltxt.read().split('\n')
testlines = testtxt.read().split('\n')
n_train = int(80*len(trainvallines)/100)
n_val = int(len(trainvallines) - n_train)
i = 0
for f in trainvallines:
    if(i<n_train):
        shutil.copy(img_path+f+".jpg",train_img)
        shutil.copy(lab_path+f+".txt",train_lab)
    else:
        shutil.copy(img_path+f+".jpg",val_img)
        shutil.copy(lab_path+f+".txt",val_lab)
    i += 1
for f in testlines:
    shutil.copy(img_path+f+".jpg","yolov5/data/number_plates_dataset/images/test")

In [ ]:
!python train.py --img 640 --batch 8 --epochs 3 --data number_plates.yaml --weights yolov5s.pt